In [16]:
import pandas as pd

In [17]:
real_news_path = "/ssddata/jimmy/temporalRAG/FNSPID/data_final_25/corpus/year_interval_selected_data_25stock.csv"
real_news = pd.read_csv(real_news_path)

In [18]:
real_news.head()

,Unnamed: 0.1,Unnamed: 0,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary,unique_id
0,97378,97378.0,2022-12-15 00:00:00 UTC,The 3 Best ETFs to Buy for 2023,AGG,https://www.nasdaq.com/articles/the-3-best-etf...,NaN,NaN,"InvestorPlace - Stock Market News, Stock Advic...",This portfolio style is more popular within th...,This portfolio style is more popular within th...,This portfolio style is more popular within th...,This portfolio style is more popular within th...,97378
1,97383,97383.0,2022-12-02 00:00:00 UTC,3 ETFs That Are All You Need for Retirement,AGG,https://www.nasdaq.com/articles/3-etfs-that-ar...,NaN,NaN,Diversification is essential to your success a...,Aggregate Bond ETF Rising interest rates have ...,Aggregate Bond ETF Rising interest rates have ...,Aggregate Bond ETF Rising interest rates have ...,Aggregate Bond ETF Rising interest rates have ...,97383
2,97386,97386.0,2022-11-16 00:00:00 UTC,Todd Rosenbluth Gets Active on Yahoo Finance,AGG,https://www.nasdaq.com/articles/todd-rosenblut...,NaN,NaN,VettaFi's head of research Todd Rosenbluth got...,Aggregate Bond ETF (AGG). VettaFi's head of re...,Aggregate Bond ETF (AGG). VettaFi's head of re...,Aggregate Bond ETF (AGG). VettaFi's head of re...,Aggregate Bond ETF (AGG). VettaFi's head of re...,97386
3,97387,97387.0,2022-11-14 00:00:00 UTC,"Don’t Get AGG-ravated, Expand Your Fixed Incom...",AGG,https://www.nasdaq.com/articles/dont-get-agg-r...,NaN,NaN,"While 2022 is not finished, two things are qui...",The following strategies are worthy of conside...,Actively-Managed Core Fixed Income ETFs The Fi...,Actively-Managed Core Fixed Income ETFs The Fi...,Actively-Managed Core Fixed Income ETFs The Fi...,97387
4,97392,97392.0,2022-09-28 00:00:00 UTC,The 10-Year Treasury Yield Just Topped 4%: Wha...,AGG,https://www.nasdaq.com/articles/the-10-year-tr...,NaN,NaN,The bear market in stocks has made portfolio v...,Inflation has prompted the Federal Reserve to ...,Aggregate Bond (NYSEMKT: AGG) and Vanguard Tot...,Inflation has prompted the Federal Reserve to ...,Inflation has prompted the Federal Reserve to ...,97392


In [19]:
import json
import pandas as pd
from datetime import datetime
import calendar
import os

def get_end_of_month_date(month_year_str):
    """
    Converts a 'Month YYYY' string (e.g., 'January 2012') to an 
    end-of-month date string in 'YYYY-MM-DD' format.
    """
    try:
        # Parse the month and year
        dt_obj = datetime.strptime(month_year_str, '%B %Y')
        # Get the last day of the month
        last_day = calendar.monthrange(dt_obj.year, dt_obj.month)[1]
        # Create a new datetime object for the end of the month
        end_of_month_dt = datetime(dt_obj.year, dt_obj.month, last_day)
        return end_of_month_dt.strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing month_year_str '{month_year_str}': {e}")
        # Fallback or raise error, here returning None or a default
        return None 


In [20]:
real_news_csv_path = real_news_path
output_dir = "/home/jimmy/Time-Aware_RAG/DQABench"

In [21]:
c_final_data = []
real_metadata_list = []
synth_metadata_list = []
global_doc_id_counter = 1

# --- Process Real News ---
print(f"Processing real news from: {real_news_csv_path}")
try:
    df_real = pd.read_csv(real_news_csv_path)
    for _, row in df_real.iterrows():
        original_id = str(row['unique_id']) # Assuming 'unique_id' is the original ID
        
        # Attempt to parse the date and format it
        try:
            original_publish_date_dt = pd.to_datetime(row['Date'])
            original_publish_date_str = original_publish_date_dt.strftime('%Y-%m-%d')
        except Exception as e:
            print(f"Warning: Could not parse date '{row['Date']}' for real news ID {original_id}. Error: {e}. Storing as is or skipping.")
            original_publish_date_str = str(row['Date']) # Store as is, or handle error appropriately

        article_text = str(row['Article'])
        article_title = str(row['Article_title']) # Keep title if needed, maybe prepend to article

        # You might want to combine title and article, e.g.:
        combined_text = f"Title:{article_title}\n{article_text}"

        global_id = f"doc_{global_doc_id_counter:05d}"

        real_metadata_list.append({
            'global_id': global_id,
            'original_id': original_id,
            'publish_date': original_publish_date_str
        })

        c_final_data.append({
            'id': global_id,
            'text': combined_text, # or combined_text
            'source_type': 'real'
        })
        global_doc_id_counter += 1
    print(f"Processed {len(df_real)} real news articles.")
except FileNotFoundError:
    print(f"Error: Real news CSV file not found at {real_news_csv_path}")
except Exception as e:
    print(f"An error occurred while processing real news: {e}")


Processing real news from: /ssddata/jimmy/temporalRAG/FNSPID/data_final_25/corpus/year_interval_selected_data_25stock.csv


Processed 23737 real news articles.


In [22]:
synth_news_json_path = "/ssddata/jimmy/temporalRAG/FNSPID/data_final_25/corpus/generated_financial_new_v2.json"

In [23]:
# --- Process Synthetic News ---
print(f"\nProcessing synthetic news from: {synth_news_json_path}")
try:
    with open(synth_news_json_path, 'r', encoding='utf-8') as f:
        synth_news_list = json.load(f)
    
    for item in synth_news_list:
        original_id = str(item['unique_id']) # UUID
        target_month_year = str(item['month_year'])
        article_text = str(item['article'])

        derived_publish_date = get_end_of_month_date(target_month_year)
        if derived_publish_date is None:
            print(f"Warning: Could not derive publish date for synthetic news ID {original_id} with month_year '{target_month_year}'. Skipping this item or using placeholder.")
            # Decide how to handle: skip, use a placeholder, or raise an error
            derived_publish_date = "N/A" # Example placeholder

        global_id = f"doc_{global_doc_id_counter:05d}"

        synth_metadata_list.append({
            'global_id': global_id,
            'original_id': original_id,
            'target_month_year': target_month_year,
            'derived_publish_date': derived_publish_date
        })

        c_final_data.append({
            'id': global_id,
            'text': article_text,
            'source_type': 'synthetic'
        })
        global_doc_id_counter += 1
    print(f"Processed {len(synth_news_list)} synthetic news articles.")
except FileNotFoundError:
    print(f"Error: Synthetic news JSON file not found at {synth_news_json_path}")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {synth_news_json_path}")
except Exception as e:
    print(f"An error occurred while processing synthetic news: {e}")

# --- Create output directory if it doesn't exist ---
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"\nCreated output directory: {output_dir}")

# --- Write C_final.jsonl ---
c_final_path = os.path.join(output_dir, 'C_final.jsonl')
print(f"\nWriting C_final.jsonl to: {c_final_path}")
try:
    with open(c_final_path, 'w', encoding='utf-8') as f:
        for entry in c_final_data:
            json.dump(entry, f)
            f.write('\n')
    print(f"Successfully wrote {len(c_final_data)} articles to C_final.jsonl")
except Exception as e:
    print(f"Error writing C_final.jsonl: {e}")

# --- Write C_real_metadata.csv ---
if real_metadata_list:
    df_real_meta = pd.DataFrame(real_metadata_list)
    real_meta_path = os.path.join("/home/jimmy/Time-Aware_RAG/DQABench_internalCheck", 'C_real_metadata.csv')
    print(f"\nWriting C_real_metadata.csv to: {real_meta_path}")
    try:
        df_real_meta.to_csv(real_meta_path, index=False, quoting=1) # quoting=1 is csv.QUOTE_ALL
        print(f"Successfully wrote {len(df_real_meta)} real news metadata entries.")
    except Exception as e:
        print(f"Error writing C_real_metadata.csv: {e}")
else:
    print("\nNo real news metadata to write.")


if synth_metadata_list:
    df_synth_meta = pd.DataFrame(synth_metadata_list)
    synth_meta_path = os.path.join("/home/jimmy/Time-Aware_RAG/DQABench_internalCheck", 'C_synth_metadata.csv')
    print(f"\nWriting C_synth_metadata.csv to: {synth_meta_path}")
    try:
        df_synth_meta.to_csv(synth_meta_path, index=False, quoting=1)
        print(f"Successfully wrote {len(df_synth_meta)} synthetic news metadata entries.")
    except Exception as e:
        print(f"Error writing C_synth_metadata.csv: {e}")
else:
    print("\nNo synthetic news metadata to write.")
    
print("\nCorpus processing complete.")



Processing synthetic news from: /ssddata/jimmy/temporalRAG/FNSPID/data_final_25/corpus/generated_financial_new_v2.json
Processed 3300 synthetic news articles.

Writing C_final.jsonl to: /home/jimmy/Time-Aware_RAG/DQABench/C_final.jsonl


Successfully wrote 27037 articles to C_final.jsonl

Writing C_real_metadata.csv to: /home/jimmy/Time-Aware_RAG/DQABench_internalCheck/C_real_metadata.csv
Successfully wrote 23737 real news metadata entries.

Writing C_synth_metadata.csv to: /home/jimmy/Time-Aware_RAG/DQABench_internalCheck/C_synth_metadata.csv
Successfully wrote 3300 synthetic news metadata entries.

Corpus processing complete.


In [24]:
# real_metadata_list_clean be the copy of real_metadata_list without "original_id" 
real_metadata_list_clean = []
for item in real_metadata_list:
    # Create a new dictionary without the 'original_id' key
    cleaned_item = {k: v for k, v in item.items() if k != 'original_id'}
    real_metadata_list_clean.append(cleaned_item)

if real_metadata_list_clean:
    df_real_meta = pd.DataFrame(real_metadata_list_clean)
    real_meta_path = os.path.join("/home/jimmy/Time-Aware_RAG/DQABench/external_metadata", 'C_real_metadata.csv')
    print(f"\nWriting C_real_metadata.csv to: {real_meta_path}")
    try:
        df_real_meta.to_csv(real_meta_path, index=False, quoting=1) # quoting=1 is csv.QUOTE_ALL
        print(f"Successfully wrote {len(df_real_meta)} real news metadata entries.")
    except Exception as e:
        print(f"Error writing C_real_metadata.csv: {e}")
else:
    print("\nNo real news metadata to write.")


Writing C_real_metadata.csv to: /home/jimmy/Time-Aware_RAG/DQABench/external_metadata/C_real_metadata.csv
Successfully wrote 23737 real news metadata entries.


In [25]:
synth_metadata_list_clean = []
for item in synth_metadata_list:
    # Create a new dictionary without the 'original_id' key
    cleaned_item = {k: v for k, v in item.items() if k != 'original_id' and k != 'target_month_year'}
    synth_metadata_list_clean.append(cleaned_item)

if synth_metadata_list_clean:
    df_synth_meta = pd.DataFrame(synth_metadata_list_clean)
    synth_meta_path = os.path.join("/home/jimmy/Time-Aware_RAG/DQABench/external_metadata", 'C_synth_metadata.csv')
    print(f"\nWriting C_synth_metadata.csv to: {synth_meta_path}")
    try:
        df_synth_meta.to_csv(synth_meta_path, index=False, quoting=1)
        print(f"Successfully wrote {len(df_synth_meta)} synthetic news metadata entries.")
    except Exception as e:
        print(f"Error writing C_synth_metadata.csv: {e}")
else:
    print("\nNo synthetic news metadata to write.")
    
print("\nCorpus processing complete.")


Writing C_synth_metadata.csv to: /home/jimmy/Time-Aware_RAG/DQABench/external_metadata/C_synth_metadata.csv
Successfully wrote 3300 synthetic news metadata entries.

Corpus processing complete.


In [30]:
id_dict = {}

for item in real_metadata_list:
    id_dict[item['original_id']] = item['global_id']

for item in synth_metadata_list:
    id_dict[item['original_id']] = item['global_id']




In [31]:
len(id_dict)

27037

In [34]:
with open("/ssddata/jimmy/temporalRAG/FNSPID/data_final_25/kb_data/corpus_combine.json", "r") as file:
    chunk_data = json.load(file)

chunk_data_output = []

for data in chunk_data:
    if data['uid'] in id_dict:
        data['corpus_id'] = id_dict[data['uid']]
        # data['chunk_uid'] = data["corpus_uid"]
        del data["corpus_uid"]
        chunk_data_output.append(data)
    else:
        print(f"ID {data['id']} not found in id_dict.")

with open("/home/jimmy/Time-Aware_RAG/experiment/extracted_time_interval.json", "w", encoding="utf-8") as file:
    json.dump(chunk_data_output, file, indent=4)